# Running Monte Carlo Transport Independently

This tutorial demonstrates how to run the Monte Carlo transport loop directly using `Simulation.from_config` without running full TARDIS iterations. This approach gives you direct control over the Monte Carlo transport process.

In [1]:
from pathlib import Path

import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.io.configuration.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import (
    initialize_estimator_statistics,
)
from tardis.transport.montecarlo.montecarlo_main_loop import (
    montecarlo_main_loop,
)
from tardis.transport.montecarlo.packets.trackers import (
    full_tracking_to_last_interaction_dataframe,
    generate_rpacket_last_interaction_tracker_list,
    generate_tracker_full_list,
    rpacket_last_interaction_tracker_list_to_dataframe,
    rpacket_trackers_to_dataframe,
    tracker_list_to_arrays,
)

/Users/wkerzend/python/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

Initializing tabulator and plotly panel extensions for widgets to work


In [2]:
# User-configurable variables
CONFIG_FILE_NAME = "tardis_example.yml"
NUMBER_OF_PACKETS = 10000
NUMBER_OF_VPACKETS = 0  # Set to 0 to disable virtual packets
ITERATION_NUMBER = 1
SHOW_PROGRESS_BARS = True
TOTAL_ITERATIONS = 1
ENABLE_RPACKET_TRACKING = False  # True: full tracking, False: last interaction only

In [3]:
# Setup simulation state from config
config_file = Path(CONFIG_FILE_NAME)
if not config_file.exists():
    raise FileNotFoundError(f"Configuration file {CONFIG_FILE_NAME} not found")

config = Configuration.from_yaml(str(config_file))
atom_data = AtomData.from_hdf("kurucz_cd23_chianti_H_He_latest.h5")
sim = Simulation.from_config(config, atom_data=atom_data)

WARNING	Task(Task-3) tardis.io.model.parse_composition_configuration:parse_composition_configuration.py:parse_density_from_config()- Number of density points larger than number of shells. Assuming inner point irrelevant
WARNING	Task(Task-3) tardis.io.model.parse_mass_fraction_configuration:parse_mass_fraction_configuration.py:parse_mass_fractions_from_config()- model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!
WARNING	Task(Task-3) tardis.io.model.parse_mass_fraction_configuration:parse_mass_fraction_configuration.py:parse_mass_fractions_from_config()- model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine depr

In [4]:
# Initialize opacity and macro atom states manually
sim.opacity_state = sim.opacity.legacy_solve(sim.plasma)

if sim.macro_atom is not None:
    sim.macro_atom_state = sim.macro_atom.solve(
        sim.plasma.j_blues,
        sim.plasma.atomic_data,
        sim.opacity_state.tau_sobolev,
        sim.plasma.stimulated_emission_factor,
        sim.opacity_state.beta_sobolev,
    )
else:
    sim.macro_atom_state = None

In [5]:
# Extract states from simulation
geometry_state = sim.simulation_state.geometry
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Initialize estimators
tau_sobolev_shape = opacity_state.tau_sobolev.shape
gamma_shape = (0, geometry_state.no_of_shells)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Convert to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(sim.macro_atom_state, line_interaction_type)

In [6]:
ENABLE_RPACKET_TRACKING = True  # Test last interaction tracker
# Create packet collection
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

# Setup packet tracking
if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_tracker_full_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
    rpacket_tracker_collection = None
else:
    # Initialize the last interaction tracker collection
    # Generate individual trackers for the main loop
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )

In [7]:
# Run the Monte Carlo main loop
v_packets_energy_hist, vpacket_tracker = (
    montecarlo_main_loop(
        packet_collection,
        geometry_state_numba,
        time_explosion,
        opacity_state_numba,
        montecarlo_configuration,
        estimators,
        spectrum_frequency_grid,
        rpacket_trackers,
        NUMBER_OF_VPACKETS,
        SHOW_PROGRESS_BARS,
    )
)


/Users/wkerzend/python/tardis/tardis/transport/montecarlo/montecarlo_main_loop.py:146: NumbaTypeSafetyWarning:

unsafe cast from uint64 to int64. Precision may be lost.



In [8]:
# Create DataFrame from tracker data
if ENABLE_RPACKET_TRACKING:
    # Full tracking: convert from rpacket_trackers list
    # Create event dataframe (all events including boundary crossings)
    tracker_df = rpacket_trackers_to_dataframe(rpacket_trackers)

    # Create last interaction dataframe from full tracking
    last_tracker_df = full_tracking_to_last_interaction_dataframe(tracker_df)

else:
    # Last interaction tracking: convert from rpacket_trackers list
    tracker_df = rpacket_last_interaction_tracker_list_to_dataframe(rpacket_trackers)

In [ ]:
# Test the new tracker_list_to_arrays function for fast array extraction
if ENABLE_RPACKET_TRACKING:
    print("Testing tracker_list_to_arrays function...")
    
    # Call the fast array extraction function
    arrays = tracker_list_to_arrays(rpacket_trackers)
    
    # Unpack the results
    (packet_id, r, shell_id, interaction_type, status,
     line_absorb_id, line_emit_id, before_nu, before_mu, before_energy,
     after_nu, after_mu, after_energy) = arrays
    
    print(f"Total events extracted: {len(packet_id)}")
    print(f"Number of packets: {len(set(packet_id))}")
    print(f"Array shapes - packet_id: {packet_id.shape}, r: {r.shape}")
    print(f"Sample packet IDs: {packet_id[:10]}")
    print(f"Sample radii: {r[:10]}")
    print(f"Sample shell IDs: {shell_id[:10]}")
    
    # Show some interaction data
    interaction_mask = ~np.isnan(before_nu)
    print(f"Number of interactions found: {interaction_mask.sum()}")
    if interaction_mask.sum() > 0:
        print(f"Sample before frequencies: {before_nu[interaction_mask][:5]}")
        print(f"Sample after frequencies: {after_nu[interaction_mask][:5]}")
    
    print("tracker_list_to_arrays function test completed!")
else:
    print("tracker_list_to_arrays only works with full tracking (ENABLE_RPACKET_TRACKING=True)")

In [9]:
tracker_df

r  shell_id interaction_type      status  \
packet_id event_id                                                        
0         0         1.235520e+15         0         BOUNDARY  IN_PROCESS   
          1         1.235520e+15         1         BOUNDARY  IN_PROCESS   
          2         1.572480e+15         2         BOUNDARY  IN_PROCESS   
          3         1.909440e+15         3         BOUNDARY  IN_PROCESS   
          4         2.246400e+15         3         BOUNDARY     EMITTED   
...                          ...       ...              ...         ...   
9999      3         1.776601e+15         1             LINE  IN_PROCESS   
          4         1.908188e+15         1             LINE  IN_PROCESS   
          5         1.908188e+15         2         BOUNDARY  IN_PROCESS   
          6         1.909440e+15         3         BOUNDARY  IN_PROCESS   
          7         2.246400e+15         3         BOUNDARY     EMITTED   

                    line_absorb_id  line_emit_id     before_nu  before_mu  \
packet_id event_id                                                          
0         0                     -1            -1           NaN        NaN   
          1                     -1            -1           NaN        NaN   
          2                     -1            -1           NaN        NaN   
          3                     -1            -1           NaN        NaN   
          4                     -1            -1           NaN        NaN   
...                            ...           ...           ...        ...   
9999      3                   9426          9426  7.783181e+14   0.562422   
          4                   9427          9427  7.730649e+14   0.407283   
          5                     -1            -1           NaN        NaN   
          6                     -1            -1           NaN        NaN   
          7                     -1            -1           NaN        NaN   

                    before_energy      after_nu  after_mu  after_energy  
packet_id event_id                                                       
0         0                   NaN           NaN       NaN           NaN  
          1                   NaN           NaN       NaN           NaN  
          2                   NaN           NaN       NaN           NaN  
          3                   NaN           NaN       NaN           NaN  
          4                   NaN           NaN       NaN           NaN  
...                           ...           ...       ...           ...  
9999      3              0.000100  7.730649e+14  0.194277      0.000098  
          4              0.000098  7.785621e+14  0.311452      0.000098  
          5                   NaN           NaN       NaN           NaN  
          6                   NaN           NaN       NaN           NaN  
          7                   NaN           NaN       NaN           NaN  

[73508 rows x 12 columns]

In [12]:
last_tracker_df

,r,last_shell_id,last_interaction_type,status,line_absorb_id,line_emit_id,before_nu,before_mu,before_energy,after_nu,after_mu,after_energy
packet_id,,,,,,,,,,,,
0,NaN,-1,NO_INTERACTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.286736e+15,0,ESCATTERING,IN_PROCESS,-1.0,-1.0,1.095839e+15,0.694482,0.000102,1.029464e+15,-0.947992,0.000102
2,1.245136e+15,0,ESCATTERING,IN_PROCESS,-1.0,-1.0,1.196269e+15,0.975195,0.000104,1.183864e+15,0.702040,0.000104
3,1.482873e+15,0,ESCATTERING,IN_PROCESS,-1.0,-1.0,7.551412e+14,0.510630,0.000100,7.706252e+14,0.956632,0.000100
4,1.544330e+15,0,ESCATTERING,IN_PROCESS,-1.0,-1.0,2.763790e+14,0.788908,0.000102,2.557440e+14,-0.906722,0.000102
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,-1,NO_INTERACTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,NaN,-1,NO_INTERACTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,1.367122e+15,0,ESCATTERING,IN_PROCESS,-1.0,-1.0,3.801189e+14,0.576656,0.000102,3.692766e+14,-0.129584,0.000102


In [33]:
last_tracker_df_create = last_tracker_df.copy()

r                        1802605583643332.25
last_shell_id                              1
last_interaction_type                   LINE
status                            IN_PROCESS
line_absorb_id                        5501.0
line_emit_id                          5501.0
before_nu                1684797044901274.75
before_mu                           0.640223
before_energy                       0.000099
after_nu                 1647734148179049.25
after_mu                             0.23445
after_energy                        0.000099
Name: 9, dtype: object

In [15]:
tracker_df.loc[9]

last_interaction_type                   LINE
before_nu                1684797044901274.75
before_mu                           0.640223
before_energy                       0.000099
after_nu                 1647734148179049.25
after_mu                             0.23445
after_energy                        0.000097
line_absorb_id                          5501
line_emit_id                            5501
interactions_count                         9
Name: 9, dtype: object

In [20]:
tracker_df.loc[9]

last_interaction_type                   LINE
before_nu                1684797044901274.75
before_mu                           0.640223
before_energy                       0.000099
after_nu                 1647734148179049.25
after_mu                             0.23445
after_energy                        0.000097
line_absorb_id                          5501
line_emit_id                            5501
interactions_count                         9
Name: 9, dtype: object